In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras.applications.vgg19 import VGG19
import numpy as np
import pandas as pd
import PIL.Image as Image
import os
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications import xception

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/BUID Dataset.zip', 'r') as zipObj:

     zipObj.extractall('BreastCancer')

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
input_folder = '/content/BreastCancer/BUID/'

In [ ]:
splitfolders.ratio(input_folder, output="dataset3",
                   seed=123, ratio=(.8, .1, .1), 
                   group_prefix=None)

Copying files: 780 files [00:00, 1593.66 files/s]


In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS = 50

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/train',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/test',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/val',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

Found 623 files belonging to 3 classes.
Found 80 files belonging to 3 classes.
Found 77 files belonging to 3 classes.


In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/dataset3/train')

class_names= train_dataset.class_names
class_names

Found 623 files belonging to 3 classes.


['benign', 'malignant', 'normal']

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
training_ds = train_ds.map(
lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
Xception1 = keras.applications.Xception(input_shape=(224,224,3),include_top=False,weights='imagenet')
Xception1.output

83697664/83683744 [==============================] - 0s 0us/step


<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'block14_sepconv2_act')>

In [ ]:
for layer in Xception1.layers:
  Xception1.trainable = False

In [ ]:
x= Flatten()(Xception1.output)
prediction=Dense(5,activation='softmax')(x)
model=Model(inputs=Xception1.input,outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
import time
start = time.time()
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)
print("Total time: ", time.time() - start, "seconds")

Epoch 1/50
10/10 [==============================] - 24s 668ms/step - loss: 45.5047 - accuracy: 0.4687 - val_loss: 23.1060 - val_accuracy: 0.5974
Epoch 2/50
10/10 [==============================] - 6s 337ms/step - loss: 19.2987 - accuracy: 0.6035 - val_loss: 22.3521 - val_accuracy: 0.6623
Epoch 3/50
10/10 [==============================] - 6s 341ms/step - loss: 8.4419 - accuracy: 0.7400 - val_loss: 19.3237 - val_accuracy: 0.6623
Epoch 4/50
10/10 [==============================] - 6s 340ms/step - loss: 4.2790 - accuracy: 0.7817 - val_loss: 14.7731 - val_accuracy: 0.6753
Epoch 5/50
10/10 [==============================] - 6s 345ms/step - loss: 1.7449 - accuracy: 0.8764 - val_loss: 17.0886 - val_accuracy: 0.7143
Epoch 6/50
10/10 [==============================] - 6s 342ms/step - loss: 0.7140 - accuracy: 0.9486 - val_loss: 13.6473 - val_accuracy: 0.7013
Epoch 7/50
10/10 [==============================] - 6s 346ms/step - loss: 0.5049 - accuracy: 0.9615 - val_loss: 15.4456 - val_accuracy: 0.6

In [ ]:
scores = model.evaluate(test_ds)

2/2 [==============================] - 1s 771ms/step - loss: 19.5165 - accuracy: 0.7125
